In [54]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from diffusers import StableDiffusion3Pipeline
import torch
from huggingface_hub import login
import os
import os
import openai


In [55]:
openai.api_key = 'sk-proj-YBEJ1hO2c9BrtzPbEp4uT3BlbkFJlgE2E9VQyWKrzTAUgmcv'

In [66]:
def get_text_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=4096):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [67]:
def generate_image_from_description(description, size="1024x1024"):
    response = openai.Image.create(
        prompt=description,
        n=1,
        size=size
    )
    return response['data'][0]['url']

In [58]:
delimiter = "####"
system_message = f"""
Summarize this text referencing quotes and details from it.\
This may include visual or auditory detail.\
The summary should also be in the same narrative format as the original text. \
The user input \
message will be delimited with {delimiter} characters.
"""
input_user_message = f"""
The sign on the wall seemed to quaver under a film of sliding warm water. Eckels felt his eyelids blink over his stare, and the sign burned in this momentary darkness:
TIME SAFARI, INC.
 SAFARIS TO ANY YEAR IN THE PAST. YOU NAME THE ANIMAL.
 WE TAKE YOU THERE.
 YOU SHOOT IT.
Warm phlegm gathered in Eckels' throat; he swallowed and pushed it down. The muscles around his mouth formed a smile as he put his hand slowly out upon the air, and in that hand waved a check for ten thousand dollars to the man behind the desk.
"Does this safari guarantee I come back alive?"
"We guarantee nothing," said the official, "except the dinosaurs." He turned. "This is Mr. Travis, your Safari Guide in the Past. He'll tell you what and where to shoot. If he says no shooting, no shooting. If you disobey instructions, there's a stiff penalty of another ten thousand dollars, plus possible government action, on your return."
Eckels glanced across the vast office at a mass and tangle, a snaking and humming of wires and steel boxes, at an aurora that flickered now orange, now silver, now blue. There was a sound like a gigantic bonfire burning all of Time, all the years and all the parchment calendars, all the hours piled high and set aflame.
A touch of the hand and this burning would, on the instant, beautifully reverse itself. Eckels remembered the wording in the advertisements to the letter. Out of chars and ashes, out of dust and coals, like golden salamanders, the old years, the green years, might leap; roses sweeten the air, white hair turn Irishblack, wrinkles vanish; all, everything fly back to seed, flee death, rush down to their beginnings, suns rise in western skies and set in glorious easts, moons eat themselves opposite to the custom, all and everything cupping one in another like Chinese boxes, rabbits into hats, all and everything returning to the fresh death, the seed death, the green death, to the time before the beginning. A touch of a hand might do it, the merest touch of a hand.
"""

# remove possible delimiters in the user's message
input_user_message = input_user_message.replace(delimiter, "")

user_message_for_model = f"""User message, \
remember that your response to the user \
must be in English: \
{delimiter}{input_user_message}{delimiter}
"""

messages =  [  
{'role':'system', 'content': system_message},    
{'role':'user', 'content': user_message_for_model},  
] 
response = get_text_from_messages(messages)
print(response)

Eckels stood in front of a sign that advertised Time Safari, Inc., offering safaris to any year in the past where you could name the animal, shoot it, and bring it back. Despite feeling uneasy, Eckels handed over a check for ten thousand dollars to the official, asking if the safari guaranteed his return. The official only guaranteed the presence of dinosaurs and introduced Mr. Travis as the Safari Guide. Eckels observed the futuristic office with wires and steel boxes, hearing a sound like a bonfire burning all of Time. He remembered the enticing advertisements promising a reversal of time with a mere touch of a hand, where everything could return to its beginning.


In [72]:
def get_summary(input_message, message_type = 0, title = None, author = None):
    added = 'This may include '
    if message_type == 0:
        added += 'notable visual and auditory detail'
    if title:
        title_author = f", {title}"
        if author:
            title_author += "by {author},"
        else:
            title_author + ','
    genre = "narrative"
    if message_type == 0:
        genre = 'storytelling'
    else:
        genre = 'narrative'
    delimiter = "####"
    system_message = f"""
    Summarize this text {title_author} referencing quotes and details from it.\
    {added}\
    The summary should also be in the same {genre} format as the original text. \
    The user input \
    message will be delimited with {delimiter} characters.
    """
    # remove possible delimiters in the user's message
    input_message = input_message.replace(delimiter, "")

    user_message_for_model = f"""
    {delimiter}{input_message}{delimiter}
    """

    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': user_message_for_model},  
    ] 
    response = get_text_from_messages(messages)
    return response

In [74]:
input_user_message = f"""
    The sign on the wall seemed to quaver under a film of sliding warm water. Eckels felt his eyelids blink over his stare, and the sign burned in this momentary darkness:
    TIME SAFARI, INC.
    SAFARIS TO ANY YEAR IN THE PAST. YOU NAME THE ANIMAL.
    WE TAKE YOU THERE.
    YOU SHOOT IT.
    Warm phlegm gathered in Eckels' throat; he swallowed and pushed it down. The muscles around his mouth formed a smile as he put his hand slowly out upon the air, and in that hand waved a check for ten thousand dollars to the man behind the desk.
    "Does this safari guarantee I come back alive?"
    "We guarantee nothing," said the official, "except the dinosaurs." He turned. "This is Mr. Travis, your Safari Guide in the Past. He'll tell you what and where to shoot. If he says no shooting, no shooting. If you disobey instructions, there's a stiff penalty of another ten thousand dollars, plus possible government action, on your return."
    Eckels glanced across the vast office at a mass and tangle, a snaking and humming of wires and steel boxes, at an aurora that flickered now orange, now silver, now blue. There was a sound like a gigantic bonfire burning all of Time, all the years and all the parchment calendars, all the hours piled high and set aflame.
    A touch of the hand and this burning would, on the instant, beautifully reverse itself. Eckels remembered the wording in the advertisements to the letter. Out of chars and ashes, out of dust and coals, like golden salamanders, the old years, the green years, might leap; roses sweeten the air, white hair turn Irishblack, wrinkles vanish; all, everything fly back to seed, flee death, rush down to their beginnings, suns rise in western skies and set in glorious easts, moons eat themselves opposite to the custom, all and everything cupping one in another like Chinese boxes, rabbits into hats, all and everything returning to the fresh death, the seed death, the green death, to the time before the beginning. A touch of a hand might do it, the merest touch of a hand.
    """

message_type = 0
get_summary(input_user_message, message_type, "A Sound of Thunder", "Ray Bradbury")

'In "A Sound of Thunder" by Ray Bradbury, Eckels visits Time Safari, Inc. where he pays for a safari to the past to hunt dinosaurs. The official warns him of strict rules and penalties for disobeying instructions. Eckels is mesmerized by the futuristic office with its humming wires and steel boxes, and the aurora that flickers in different colors. He contemplates the power to change history with a simple touch, imagining a world where time reverses itself and everything returns to its beginnings. The vivid descriptions create a sense of wonder and danger as Eckels prepares to embark on his journey to the past.'

In [70]:
def get_image(input_message, message_type = 0, title = None, author = None):
    response = generate_image_from_description(description = input_message)
    return response

In [73]:
get_image(get_summary(input_user_message, message_type, "A Sound of Thunder", "Ray Bradbury"))

'https://oaidalleapiprodscus.blob.core.windows.net/private/org-XSkwm73bL0mx6bBBPlcZAGhH/user-PRpqn32bfoZlh94TediQwFwa/img-GPMn6PL5SOViCtcFXY2CHDzk.png?st=2024-06-22T21%3A46%3A49Z&se=2024-06-22T23%3A46%3A49Z&sp=r&sv=2023-11-03&sr=b&rscd=inline&rsct=image/png&skoid=6aaadede-4fb3-4698-a8f6-684d7786b067&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2024-06-21T23%3A40%3A23Z&ske=2024-06-22T23%3A40%3A23Z&sks=b&skv=2023-11-03&sig=P1z28iNAF5nwirvXWdgrHs9FtcV/tgd5eU/SN8x5G00%3D'